In [57]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.optimize import minimize


In [58]:
# Example data (modify with actual values from the thesis)
num_recipes = 5
batch_sizes = [100, 200, 150, 80, 120]  # Number of PLBs per recipe
tact_time = 8  # Seconds between PLBs entering the line

# Processing times per recipe per machine (in minutes)
# Format: {machine: [time_recipe_1, time_recipe_2, ...]}
processing_times = {
    "DoughDis": [5, 6, 4, 7, 5],
    "Climas": [120, 110, 130, 115, 125],
    "toOven": [10, 12, 8, 15, 9],
    "Oven": [25, 20, 30, 22, 28],
    "CoolFreeze": [60, 55, 65, 58, 62]
}

# Change-over times between recipes (sequence-dependent)
# Format: {machine: [[time_r1_to_r1, time_r1_to_r2, ...], ...]}
changeover_times = {
    "DoughDis": np.random.randint(5, 15, (num_recipes, num_recipes)),
    "Climas": np.random.randint(5, 15, (num_recipes, num_recipes)),
    "toOven": np.random.randint(5, 15, (num_recipes, num_recipes)),
    "Oven": np.random.randint(5, 15, (num_recipes, num_recipes)),
    "CoolFreeze": np.random.randint(5, 15, (num_recipes, num_recipes)),
}

In [59]:
print("processing_times:\n", processing_times)

print("Changeover times:\n", changeover_times)

processing_times:
 {'DoughDis': [5, 6, 4, 7, 5], 'Climas': [120, 110, 130, 115, 125], 'toOven': [10, 12, 8, 15, 9], 'Oven': [25, 20, 30, 22, 28], 'CoolFreeze': [60, 55, 65, 58, 62]}
Changeover times:
 {'DoughDis': array([[12,  6,  7,  5,  5],
       [ 7,  9,  7,  5,  5],
       [12, 14,  6,  7,  6],
       [ 7, 11,  5, 14, 12],
       [14, 14, 14,  6,  7]]), 'Climas': array([[13, 11,  8, 14,  9],
       [ 6, 12,  8, 13,  9],
       [13,  8, 14,  9, 13],
       [12,  7,  5,  7,  8],
       [ 6,  5, 11, 12, 11]]), 'toOven': array([[ 9,  5, 11, 11, 13],
       [ 7, 13,  5,  5,  8],
       [13, 10,  7,  5,  8],
       [13,  7, 13, 11,  8],
       [ 7, 14,  9,  9,  7]]), 'Oven': array([[13,  8,  9,  8,  9],
       [11, 13, 11,  9, 14],
       [14, 11, 14,  9,  7],
       [11,  6, 13, 14, 14],
       [ 5, 10, 11, 12, 14]]), 'CoolFreeze': array([[13,  6, 14,  6,  9],
       [ 9, 10,  7, 12,  5],
       [10,  8,  5, 11, 13],
       [ 8,  8, 10,  7, 10],
       [11, 14, 14,  7, 11]])}


In [60]:
changeover_times["DoughDis"][1, 2]

7

In [61]:
class BakerySchedulingProblem(ElementwiseProblem):
    def __init__(self, batch_sizes, processing_times, changeover_times, tact_time):
        super().__init__(n_var=num_recipes, n_obj=1, xl=0, xu=num_recipes-1, vtype=int)
        self.batch_sizes = batch_sizes
        self.processing_times = processing_times
        self.changeover_times = changeover_times
        self.tact_time = tact_time
        self.machines = list(processing_times.keys())

    def _evaluate(self, x, out, *args, **kwargs):
        # x is a permutation of recipe indices (e.g., [2, 0, 3, 1, 4])
        sequence = x.astype(int)
        makespan = self.calculate_makespan(sequence)
        out["F"] = makespan

    def calculate_makespan(self, sequence):
        # Initialize completion times for all machines and recipes
        num_machines = len(self.machines)
        completion_times = np.zeros((num_machines, len(sequence)))


        for i, recipe in enumerate(sequence):
            for m_idx, machine in enumerate(self.machines):
                # Processing time for this recipe on current machine
                proc_time = self.processing_times[machine][recipe]
                batch_size = self.batch_sizes[recipe]
                time_per_batch = proc_time + (batch_size - 1) * self.tact_time / 60  # Convert to minutes

                # Change-over time (previous recipe to current recipe)
                if i == 0:
                    changeover = 0
                else:
                    prev_recipe = sequence[i-1]
                    changeover = self.changeover_times[machine][prev_recipe, recipe]

                # Start time is max of:
                # 1. Previous recipe's completion on this machine + changeover
                # 2. Current recipe's completion on previous machine
                if m_idx == 0:
                    start_time = (completion_times[m_idx, i-1] + changeover) if i > 0 else 0
                else:
                    start_time = max(
                        completion_times[m_idx-1, i],  # Completion on previous machine
                        (completion_times[m_idx, i-1] + changeover) if i > 0 else 0
                    )

                completion_times[m_idx, i] = start_time + time_per_batch

        return completion_times[-1, -1]  # Makespan is last machine's last completion time

In [62]:
problem = BakerySchedulingProblem(batch_sizes, processing_times, changeover_times, tact_time)

algorithm = GA(
    pop_size=50,
    crossover=OrderCrossover(),
    mutation=InversionMutation(),
    eliminate_duplicates=True
)

res = minimize(
    problem,
    algorithm,
    ("n_gen", 100),
    seed=42,
    verbose=True
)

print("Best solution:", res.X)
print("Makespan (minutes):", res.F)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       50 |  8.915733E+02 |  8.168000E+02


ValueError: could not broadcast input array from shape (7,) into shape (5,)